# Assignment 2

This assignment is about training and evaluating a POS tagger with some real data. The dataset is available through the Universal Dependencies (https://universaldependencies.org/) (UD) project. To get to know the project, please visit https://universaldependencies.org/introduction.html)

In [1]:
! pip install conllutils
! pip install conllu
! pip install conll-df


In [2]:
import numpy as np
import pandas as pd
import operator
import nltk

import conllutils
from io import open
from conllu import parse_incr

from collections import defaultdict
from conllutils import pipe
from conll_df import conll_df



**Part 1** (getting the data)

You can download the dataset files directly from the UD website, but it will let you only download all the languages in one compressed file. In this assignment you will be working with th GUM dataset, which you can download directly from:
https://github.com/UniversalDependencies/UD_English-GUM.
Please download it to your colab machine.



In [3]:
!git clone https://github.com/UniversalDependencies/UD_English-GUM

fatal: destination path 'UD_English-GUM' already exists and is not an empty directory.


We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. 
You may choose the data structure by yourself. 
As you can see
1. every word is represented by a line
2. columns representing specific features. 
   * We are only interested in the first and fourth columns 
   * corresponding to the word and its POS tag.

### Set Path's

In [4]:
# Your code goes here
FOLDER = 'UD_English-GUM'



user = 'va'
if user == 'Or':
    ud_dev =   r"C:\MSC\NLP2\HW2\UD_English-GUM\en_gum-ud-dev.conllu"
    ud_train = r"C:\MSC\NLP2\HW2\UD_English-GUM\en_gum-ud-train.conllu"
    ud_test =  r"C:\MSC\NLP2\HW2\UD_English-GUM\en_gum-ud-test.conllu"
elif user == 'Roni':
    ud_dev =   '/Users/ronibendom/Master/NLP/HW2/UD_English-GUM/en_gum-ud-dev.conllu'
    ud_train = '/Users/ronibendom/Master/NLP/HW2/UD_English-GUM/en_gum-ud-train.conllu'
    ud_test =  '/Users/ronibendom/Master/NLP/HW2/UD_English-GUM/en_gum-ud-test.conllu'
else:
    ud_dev =   f'{FOLDER}/en_gum-ud-dev.conllu'
    ud_train = f'{FOLDER}/en_gum-ud-train.conllu'
    ud_test =  f'{FOLDER}/en_gum-ud-test.conllu'

train_csv = FOLDER + '/en_gum-ud-train.csv'
test_csv = FOLDER + '/en_gum-ud-test.csv'
dev_csv = FOLDER + '/en_gum-ud-dev.csv'


### Get Data

In [5]:
train_df = conll_df(ud_train, file_index=False)
train_df = train_df.iloc[:, [0, 3]]

dev_df = conll_df(ud_dev, file_index=False)
dev_df = dev_df.iloc[:, [0, 3]]

test_df = conll_df(ud_test, file_index=False)
test_df = test_df.iloc[:, [0, 3]]

# train_df.to_csv(train_csv)
# test_df.to_csv(test_csv)
# dev_df.to_csv(dev_csv)

In [6]:

# train_df = pd.read_csv(train_csv)
# dev_df = pd.read_csv(dev_csv)
# test_df = pd.read_csv(test_csv)

In [7]:
def extract_ommision_matrix_B(train_df, unique_pos, words_indexing_dict):

    B = np.zeros([len(unique_pos), len(words_indexing_dict)])
    B_row_index = 0
    for i_pos in unique_pos:
        i_pos_train_df = train_df.loc[train_df['p'] == i_pos]
        i_pos_words, i_pos_word_count = np.unique(i_pos_train_df.loc[:, 'w'].values, return_counts=True)
        i_pos_percent = i_pos_word_count / np.sum(i_pos_word_count)
        for i_word in i_pos_words:
            updated_percent_per_word_per_pos = i_pos_percent[np.where(i_pos_words == i_word)[0][0]]
            B_column_word_index = words_indexing_dict[i_word]
            B[B_row_index, B_column_word_index] = updated_percent_per_word_per_pos
        B_row_index += 1

    return B

def generate_transition_matrix_A_and_pi(train_df, unique_pos, pos_indexing_dict):
    A = np.zeros((len(unique_pos), len(unique_pos)))
    pi = np.zeros(([len(unique_pos), 1]))

    sentence_ind = 1
    pi[pos_indexing_dict[train_df.iloc[0, :]['p']]] += 1

    for i in range(1, train_df.shape[0]):
        curr_sentence_ind = train_df.index[i][0]
        if curr_sentence_ind != sentence_ind:
            pi[pos_indexing_dict[train_df.iloc[i, :]['p']]] += 1
            sentence_ind += 1
        else:
            A[pos_indexing_dict[train_df.iloc[i-1, :]['p']], pos_indexing_dict[train_df.iloc[i, :]['p']]] += 1
    
    A = A/A.sum(axis=1, keepdims=True)
    pi = pi / sum(pi)

    return A, pi

### Create matrices

In [8]:
### Create matrices
pos_values = list(np.unique(train_df.loc[:, 'p'].values, return_counts=True))
unique_words = np.unique(train_df.loc[:, 'w'].values)
unique_pos = pos_values[0]

words_indexing_dict = {unique_words[i] : i for i in range(len(unique_words))}
pos_indexing_dict = {unique_pos[i] : i for i in range(len(unique_pos))}

# Generate omissiom matrix
B = extract_ommision_matrix_B(train_df, unique_pos, words_indexing_dict)

# Generate transition matrix and initial marix
A, pi_initial_matrix = generate_transition_matrix_A_and_pi(train_df, unique_pos, pos_indexing_dict)

### Create sentences from df

In [10]:
def sentences_from_df(df):
    sentences = []
    sentence_ind = 1
    sentence = []

    for i in range(df.shape[0]):
        curr_sentence_ind = df.index[i][0]
        if curr_sentence_ind != sentence_ind:
            sentences.append(str.join(" ", sentence))
            sentence_ind += 1
            sentence = []
        sentence.append(df.iloc[i, :]['w'])

    return sentences

In [11]:
sentences = sentences_from_df(train_df)

**Part 2**

Write a class **simple_tagger**
1. with methods *train* and *evaluate*. 
2. The method *train* receives the data as a list of sentences
3. use it for training the tagger.
4. In this case, it should learn a simple dictionary that maps words to tags
    * defined as the most frequent tag for every word (in case there is more than one most frequent tag, you may select one of them randomly).
    * The dictionary should be stored as a class member for evaluation.

The method *evaluate* 
1. receives the data as a list of sentences
2. use it to evaluate the tagger performance. 
3. Specifically, you should calculate the word and sentence level accuracy.
4. The evaluation process is simply going word by word
5. querying the dictionary (created by the train method) for each word’s tag and compare it to the true tag of that word. 
6. The word-level accuracy is the number of successes divided by the number of words.
7. For OOV (out of vocabulary, or unknown) words
8. the tagger should assign the most frequent tag in the entire training set (i.e., the mode).
9. The function should return the two numbers:
    * word level accuracy
    * sentence level accuracy.


Write a class simple_tagger, with methods train and evaluate. The method train receives the data as a list of sentences, and use it for training the tagger. In this case, it should learn a simple dictionary that maps words to tags, defined as the most frequent tag for every word (in case there is more than one most frequent tag, you may select one of them randomly). The dictionary should be stored as a class member for evaluation.

The method evaluate receives the data as a list of sentences, and use it to evaluate the tagger performance. Specifically, you should calculate the word and sentence level accuracy. The evaluation process is simply going word by word, querying the dictionary (created by the train method) for each word’s tag and compare it to the true tag of that word. The word-level accuracy is the number of successes divided by the number of words. For OOV (out of vocabulary, or unknown) words, the tagger should assign the most frequent tag in the entire training set (i.e., the mode). The function should return the two numbers: word level accuracy and sentence level accuracy.

In [ ]:
def get_list_of_sentences_tag_lists(df):
    word_index_array = df['i'].to_numpy()
    initial_sentence_idx = np.where(word_index_array == 1)[0]
    words_list = df['w'].to_list()
    tag_list = df['p'].to_list()

    sentence_list  = (initial_sentence_idx.size)*['None']
    sentence_tag_list  = (initial_sentence_idx.size)*['None']

    for sentence_idx in range(initial_sentence_idx.size):
        if not sentence_idx == initial_sentence_idx.size-1:
            
            curr_sentence =  words_list[initial_sentence_idx[sentence_idx]:initial_sentence_idx[sentence_idx+1]]
            curr_tag =  tag_list[initial_sentence_idx[sentence_idx]:initial_sentence_idx[sentence_idx+1]]
        else:
            curr_sentence =  words_list[initial_sentence_idx[sentence_idx]::]
            curr_tag =  tag_list[initial_sentence_idx[sentence_idx]::]
        
        sentence_list[sentence_idx] = curr_sentence
        sentence_tag_list[sentence_idx] = curr_tag

    return sentence_list, sentence_tag_list

In [20]:

class simple_tagger:
    def __init__(self):
        self.tagger = {}
    
    def train(self, data):
        pos = np.unique(data['p'], return_counts=True)
        most_frequent_pos = pos[0][np.where(pos[1] == max(pos[1]))][0]

        tagger = defaultdict(lambda: most_frequent_pos)

        unique_words = np.unique(data.loc[:, 'w'].values)
        for word in unique_words:
            word_data = data.loc[data['w'] == word]
            word_data_count = word_data['p'].value_counts()
            predicted_tag = word_data_count.index[0]
            # predicted_tag = pos_of_word = np.unique(data.loc[data['w'] == word, 'p'], return_counts=True)
            tagger[word] = predicted_tag

        self.tagger = tagger
    def evaluate(self, data):
        # sentences = sentences_from_df(data)
        sentences_list, sentence_tag_list = get_list_of_sentences_tag_lists(data)

        words_success = 0
        sentences_success = 0
        idx = pd.IndexSlice
        for sentence_num, (sentence, sentence_tags) in enumerate(zip(sentences_list, sentence_tag_list)):
        # for sentence_num, sentence in enumerate(sentences, 1):
            count_successes = 0
            # sentence_df = data.loc[idx[sentence_num], 'p'].values
            for word_num, (word, actual_tag) in enumerate(zip(sentence, sentence_tags)):
            # for word_num, word in enumerate(sentence.split(' ')):
                predicted_tag = self.tagger[word]
                # actual_tag = sentence_df[word_num]

                if predicted_tag == actual_tag:
                    words_success += 1
                    count_successes += 1

            if count_successes == len(sentence)-1:
                sentences_success += 1

        word_accuracy = round(words_success / data.shape[0] * 100, 4)
        sentence_accuracy = round(sentences_success / len(sentences_list) * 100, 4)

        return word_accuracy, sentence_accuracy
    
    

In [21]:
tagger = simple_tagger()
tagger.train(train_df)

In [22]:
simple_tagger_word_accuracy_train, simple_tagger_sentence_accuracy_train = tagger.evaluate(train_df)
simple_tagger_word_accuracy_dev, simple_tagger_sentence_accuracy_dev = tagger.evaluate(dev_df)
simple_tagger_word_accuracy_test, simple_tagger_sentence_accuracy_test = tagger.evaluate(test_df)


In [24]:


result_list = [['train' ,simple_tagger_word_accuracy_train, simple_tagger_sentence_accuracy_train],
['dev', simple_tagger_word_accuracy_dev, simple_tagger_sentence_accuracy_dev],
['test', simple_tagger_word_accuracy_test, simple_tagger_sentence_accuracy_test]]

simple_tagger_results =  pd.DataFrame(result_list, columns = ['data-set', 'word-accuracy[%]', 'sentence-accuracy[%]'])
simple_tagger_results.head()

train data: word accuracy = 93.0855%, sentence accuracy = 36.9323%
dev data: word accuracy = 84.8186%, sentence accuracy = 16.8646%
test data: word accuracy = 82.4505%, sentence accuracy = 16.0134%


**Part 3**

Similar to part 2, write the class hmm_tagger, which implements HMM tagging. The method *train* should build the matrices A, B and Pi, from the data as discussed in class. The method *evaluate* should find the best tag sequence for every input sentence using he Viterbi decoding algorithm, and then calculate the word and sentence level accuracy using the gold-standard tags. You should implement the Viterbi algorithm in the next block and call it from your class.

Additional guidance:
1. The matrix B represents the emissions probabilities. Since B is a matrix, you should build a dictionary that maps every unique word in the corpus to a serial numeric id (starting with 0). This way columns in B represents word ids.
2. During the evaluation, you should first convert each word into it’s index and then create the observation array to be given to Viterbi, as a list of ids. OOV words should be assigned with a random tag. To make sure Viterbi works appropriately, you can simply break the sentence into multiple segments every time you see an OOV word, and decode every segment individually using Viterbi.


In [ ]:
def dptable(V):
    yield " ".join(("%12d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)

def viterbi_algorithm(observations, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
         V[0][st] = {"prob": start_p[st] * emit_p[st][observations[0]], "prev": None}
   
    for t in range(1, len(observations)):
        V.append({})
        for st in states:
            max_tr_prob = V[t - 1][states[0]]["prob"] * trans_p[states[0]][st]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t - 1][prev_st]["prob"] * trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
 
            max_prob = max_tr_prob * emit_p[st][observations[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}
     
    # for line in dptable(V):
    #    print(line)
    opt = []
    max_prob = 0.0
    best_st = None
 
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st
 
 
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]
        
    return opt

In [ ]:
   
class hmm_tagger:
    data = pd.DataFrame()
    A = []
    B = []
    Pi = []
    uniqueWordsArr = []
    
    def __init__(self, data_df):
        
        unique_pos, pos_counts = list(np.unique(data_df.loc[:, 'p'].values, return_counts=True))
        unique_words, word_counts = np.unique(data_df.loc[:, 'w'].values, return_counts=True)
        unique_words_list  = unique_words.tolist()
        unique_pos_list  = unique_pos.tolist()
        
        self.data_df = data_df
        self.unique_pos = unique_pos
        self.unique_words = unique_words
        self.unique_words_list = unique_words_list
        self.unique_pos_list = unique_pos_list


    
    def train(self):

        data_df = self.data_df
        unique_pos = self.unique_pos
        unique_words = self.unique_words
        
        
        B = extract_ommision_matrix_B(self.data_df, unique_pos, unique_words)
        A = generate_transition_matrix_A(self.data_df, unique_pos, unique_words)
        Pi = np.ones([unique_pos.size,1])*(1/unique_pos.size)

        B = pd.DataFrame(B.T, index = unique_words, columns = unique_pos)
        A = pd.DataFrame(A.T, index =unique_pos , columns = unique_pos)
        Pi = pd.DataFrame(Pi.T, columns = unique_pos)
   
        self.states = tuple(unique_pos)
        self.A = A.to_dict()
        self.B = B.to_dict()
        self.Pi = Pi.to_dict('records')[0]
        
        # self.A, self.B, self.Pi = calc_A_B_Pi(data_df)
        self.uniqueWordsArr = unique_words.tolist()

   
    
    def evaluate(self, data):
        # sentences = sentences_from_df(data)
        sentences_list, sentence_tag_list = get_list_of_sentences_tag_lists(data)
        sentenceIdx = [] 
        words_success = 0
        sentences_success = 0
        idx = pd.IndexSlice
        for sentence_num, (sentence, sentence_tags) in enumerate(zip(sentences_list, sentence_tag_list)):
            # for sentence_num, sentence in enumerate(sentences, 1):
            count_successes = 0
            
            
            # validate all word in sentence is in train data set otherwise
            # choose randomly word from bank of word
            for i_word_idx, word in enumerate(sentence):
                if not word in self.unique_words:
                    sentence[i_word_idx] = random.choice(self.unique_words_list)
                
                
            predicted_tag_list = viterbi_algorithm(tuple(sentence), self.states, self.Pi, self.A, self.B)  
            
            for word_num, (predicted_tag, actual_tag) in enumerate(zip(predicted_tag_list, sentence_tags)):
                # print('word nun ' + str(word_num))

                if predicted_tag == actual_tag:
                    words_success += 1
                    count_successes += 1

            if count_successes == len(sentence)-1:
                sentences_success += 1

        word_accuracy = round(words_success / data.shape[0] * 100, 4)
        sentence_accuracy = round(sentences_success / len(sentences_list) * 100, 4)

        return word_accuracy, sentence_accuracy


In [ ]:

hmmTagger = hmm_tagger(train_df)
hmmTagger.train()

hmm_word_level_accuracy_train, hmm_sentence_level_accuracy_train = hmmTagger.evaluate(train_df)
hmm_word_level_accuracy_test, hmm_sentence_level_accuracy_test = hmmTagger.evaluate(test_df)
hmm_word_level_accuracy_dev, hmm_sentence_level_accuracy_dev = hmmTagger.evaluate(dev_df)


In [ ]:


result_list = [['train' ,hmm_word_level_accuracy_train, hmm_sentence_level_accuracy_train],
['dev', hmm_word_level_accuracy_dev, hmm_sentence_level_accuracy_dev],
['test', hmm_word_level_accuracy_test, hmm_sentence_level_accuracy_test]]

simple_tagger_results =  pd.DataFrame(result_list, columns = ['data-set', 'word-accuracy[%]', 'sentence-accuracy[%]'])
simple_tagger_results.head()

**Part 4**

Compare the results obtained from both taggers and a MEMM tagger, implemented by NLTK (a known NLP library), over both, the dev and test datasets. To train the NLTK MEMM tagger you should execute the following lines (it may take some time to train...):

In [ ]:
from nltk.tag import tnt 

tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_data)
print(tnt_pos_tagger.evaluate(test_data))

Print both, word level and sentence level accuracy for all the three taggers in a table.

In [ ]:
# Your code goes here